In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession


In [3]:
spark = SparkSession.builder.appName('hack-clustering').getOrCreate()

In [4]:
data = sqlContext.sql("SELECT * FROM hack_data_csv")

In [5]:
data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+----+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted| Location|WPM_Typing_Speed|hack|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+----+
 8.0| 391.09| 1| 2.96| 7.0| Slovenia| 72.37| 1|
 20.0| 720.99| 0| 3.04| 9.0|British Virgin Is...| 69.08| 1|
 31.0| 356.32| 1| 3.71| 8.0| Tokelau| 70.58| 1|
 2.0| 228.08| 1| 2.48| 8.0| Bolivia| 70.8| 1|
 20.0| 408.5| 0| 3.57| 8.0| Iraq| 71.28| 1|
 1.0| 390.69| 1| 2.79| 9.0| Marshall Islands| 71.57| 1|
 18.0| 342.97| 1| 5.1| 7.0| Georgia| 72.32| 1|
 22.0| 101.61| 1| 3.03| 7.0| Timor-Leste| 72.03| 1|
 15.0| 275.53| 1| 3.53| 8.0|Palestinian Terri...| 70.17| 1|
 12.0| 424.83| 1| 2.53| 8.0| Bangladesh| 69.99| 1|
 15.0| 249.09| 1| 3.39| 9.0|Northern Mariana ...| 70.77| 1|
 32.0| 242.48| 0| 4.24| 8.0| Zimbabwe| 67.93| 1|
 23.0| 514.54| 0| 3.18| 8.0| Isle of Man| 68.56| 1|
 9.0| 284.77| 0| 3.12| 9.0|Sao Tome and Prin...| 70.82| 1|
 27.0| 779.25| 1| 2.37| 8.0| Greece| 72.73| 1|
 12.0| 307.31| 1| 3.22| 7.0| Solomon Islands| 67.95| 1|
 21.0| 355.94| 1| 2.0| 7.0| Guinea-Bissau| 72.0| 1|
 10.0| 372.65| 0| 3.33| 7.0| Burkina Faso| 69.19| 1|
 20.0| 347.23| 1| 2.33| 7.0| Mongolia| 70.41| 1|
 22.0| 456.57| 0| 1.52| 8.0| Nigeria| 69.35| 1|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+----+
only showing top 20 rows

In [6]:
data.head()

Out[ 9 ]: Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location=u'Slovenia', WPM_Typing_Speed=72.37, hack=1)

In [7]:
cols = ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [8]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

In [9]:
assembler = VectorAssembler(inputCols=cols, outputCol='features')


In [10]:
assembled_data = assembler.transform(data)

In [11]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

In [12]:
scaler_model = scaler.fit(assembled_data)
scaled_data = scaler_model.transform(assembled_data)
scaled_data.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- Location: string (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)
-- hack: integer (nullable = true)
-- features: vector (nullable = true)
-- scaledFeatures: vector (nullable = true)

In [13]:
k_means_2 = KMeans(featuresCol='scaledFeatures', k=2)
k_means_3 = KMeans(featuresCol='scaledFeatures', k=3)

In [14]:
model_k2 = k_means_2.fit(scaled_data)
model_k3 = k_means_3.fit(scaled_data)

In [15]:
model_k3_data = model_k3.transform(scaled_data)
model_k3_data.groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 2| 79|
 0| 88|
+----------+-----+

In [16]:
model_k2_data = model_k2.transform(scaled_data)
model_k2_data.groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+

In [17]:
assembled_data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+----+--------------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted| Location|WPM_Typing_Speed|hack| features|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+----+--------------------+
 8.0| 391.09| 1| 2.96| 7.0| Slovenia| 72.37| 1|[8.0,391.09,1.0,2...|
 20.0| 720.99| 0| 3.04| 9.0|British Virgin Is...| 69.08| 1|[20.0,720.99,0.0,...|
 31.0| 356.32| 1| 3.71| 8.0| Tokelau| 70.58| 1|[31.0,356.32,1.0,...|
 2.0| 228.08| 1| 2.48| 8.0| Bolivia| 70.8| 1|[2.0,228.08,1.0,2...|
 20.0| 408.5| 0| 3.57| 8.0| Iraq| 71.28| 1|[20.0,408.5,0.0,3...|
 1.0| 390.69| 1| 2.79| 9.0| Marshall Islands| 71.57| 1|[1.0,390.69,1.0,2...|
 18.0| 342.97| 1| 5.1| 7.0| Georgia| 72.32| 1|[18.0,342.97,1.0,...|
 22.0| 101.61| 1| 3.03| 7.0| Timor-Leste| 72.03| 1|[22.0,101.61,1.0,...|
 15.0| 275.53| 1| 3.53| 8.0|Palestinian Terri...| 70.17| 1|[15.0,275.53,1.0,...|
 12.0| 424.83| 1| 2.53| 8.0| Bangladesh| 69.99| 1|[12.0,424.83,1.0,...|
 15.0| 249.09| 1| 3.39| 9.0|Northern Mariana ...| 70.77| 1|[15.0,249.09,1.0,...|
 32.0| 242.48| 0| 4.24| 8.0| Zimbabwe| 67.93| 1|[32.0,242.48,0.0,...|
 23.0| 514.54| 0| 3.18| 8.0| Isle of Man| 68.56| 1|[23.0,514.54,0.0,...|
 9.0| 284.77| 0| 3.12| 9.0|Sao Tome and Prin...| 70.82| 1|[9.0,284.77,0.0,3...|
 27.0| 779.25| 1| 2.37| 8.0| Greece| 72.73| 1|[27.0,779.25,1.0,...|
 12.0| 307.31| 1| 3.22| 7.0| Solomon Islands| 67.95| 1|[12.0,307.31,1.0,...|
 21.0| 355.94| 1| 2.0| 7.0| Guinea-Bissau| 72.0| 1|[21.0,355.94,1.0,...|
 10.0| 372.65| 0| 3.33| 7.0| Burkina Faso| 69.19| 1|[10.0,372.65,0.0,...|
 20.0| 347.23| 1| 2.33| 7.0| Mongolia| 70.41| 1|[20.0,347.23,1.0,...|
 22.0| 456.57| 0| 1.52| 8.0| Nigeria| 69.35| 1|[22.0,456.57,0.0,...|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+----+--------------------+
only showing top 20 rows

In [18]:
import matplotlib.pyplot as plt


In [19]:
import seaborn as sns

In [20]:
sns.set_style('whitegrid')

In [21]:
scaled_data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+----+--------------------+--------------------+
Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted| Location|WPM_Typing_Speed|hack| features| scaledFeatures|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+----+--------------------+--------------------+
 8.0| 391.09| 1| 2.96| 7.0| Slovenia| 72.37| 1|[8.0,391.09,1.0,2...|[0.56785108466505...|
 20.0| 720.99| 0| 3.04| 9.0|British Virgin Is...| 69.08| 1|[20.0,720.99,0.0,...|[1.41962771166263...|
 31.0| 356.32| 1| 3.71| 8.0| Tokelau| 70.58| 1|[31.0,356.32,1.0,...|[2.20042295307707...|
 2.0| 228.08| 1| 2.48| 8.0| Bolivia| 70.8| 1|[2.0,228.08,1.0,2...|[0.14196277116626...|
 20.0| 408.5| 0| 3.57| 8.0| Iraq| 71.28| 1|[20.0,408.5,0.0,3...|[1.41962771166263...|
 1.0| 390.69| 1| 2.79| 9.0| Marshall Islands| 71.57| 1|[1.0,390.69,1.0,2...|[0.07098138558313...|
 18.0| 342.97| 1| 5.1| 7.0| Georgia| 72.32| 1|[18.0,342.97,1.0,...|[1.27766494049636...|
 22.0| 101.61| 1| 3.03| 7.0| Timor-Leste| 72.03| 1|[22.0,101.61,1.0,...|[1.56159048282889...|
 15.0| 275.53| 1| 3.53| 8.0|Palestinian Terri...| 70.17| 1|[15.0,275.53,1.0,...|[1.06472078374697...|
 12.0| 424.83| 1| 2.53| 8.0| Bangladesh| 69.99| 1|[12.0,424.83,1.0,...|[0.85177662699757...|
 15.0| 249.09| 1| 3.39| 9.0|Northern Mariana ...| 70.77| 1|[15.0,249.09,1.0,...|[1.06472078374697...|
 32.0| 242.48| 0| 4.24| 8.0| Zimbabwe| 67.93| 1|[32.0,242.48,0.0,...|[2.27140433866020...|
 23.0| 514.54| 0| 3.18| 8.0| Isle of Man| 68.56| 1|[23.0,514.54,0.0,...|[1.63257186841202...|
 9.0| 284.77| 0| 3.12| 9.0|Sao Tome and Prin...| 70.82| 1|[9.0,284.77,0.0,3...|[0.63883247024818...|
 27.0| 779.25| 1| 2.37| 8.0| Greece| 72.73| 1|[27.0,779.25,1.0,...|[1.91649741074455...|
 12.0| 307.31| 1| 3.22| 7.0| Solomon Islands| 67.95| 1|[12.0,307.31,1.0,...|[0.85177662699757...|
 21.0| 355.94| 1| 2.0| 7.0| Guinea-Bissau| 72.0| 1|[21.0,355.94,1.0,...|[1.49060909724576...|
 10.0| 372.65| 0| 3.33| 7.0| Burkina Faso| 69.19| 1|[10.0,372.65,0.0,...|[0.70981385583131...|
 20.0| 347.23| 1| 2.33| 7.0| Mongolia| 70.41| 1|[20.0,347.23,1.0,...|[1.41962771166263...|
 22.0| 456.57| 0| 1.52| 8.0| Nigeria| 69.35| 1|[22.0,456.57,0.0,...|[1.56159048282889...|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+----+--------------------+--------------------+
only showing top 20 rows

In [22]:
import matplotlib.pyplot as plt

In [23]:
pdf = scaled_data.toPandas()

In [24]:
plt.scatter('scaledFeatures','hack',data=pdf)

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-1125850033132121> in <module> () 
 ----> 1 plt . scatter ( 'scaledFeatures' , 'hack' , data = pdf ) 

 /databricks/python/local/lib/python2.7/site-packages/matplotlib/pyplot.pyc in scatter (x, y, s, c, marker, cmap, norm, vmin, vmax, alpha, linewidths, verts, edgecolors, hold, data, **kwargs) 
 3256 vmin = vmin , vmax = vmax , alpha = alpha , 
 3257 linewidths = linewidths , verts = verts , 
 -> 3258 edgecolors=edgecolors, data=data, **kwargs)
 3259 finally : 
 3260 ax . hold ( washold ) 

 /databricks/python/local/lib/python2.7/site-packages/matplotlib/__init__.pyc in inner (ax, *args, **kwargs) 
 1817 warnings.warn(msg % (label_namer, func.__name__),
 1818 RuntimeWarning, stacklevel=2)
 -> 1819 return func ( ax , * args , ** kwargs ) 
 1820 pre_doc = inner . __doc__ 
 1821 if pre_doc is None : 

 /databricks/python/local/lib/python2.7/site-packages/matplotlib/axes/_axes.pyc in scatter (self, x, y, s, c, marker, cmap, norm, vmin, vmax, alpha, linewidths, verts, edgecolors, **kwargs) 
 3893 self . set_ymargin ( 0.05 ) 
 3894 
 -> 3895 self . add_collection ( collection ) 
 3896 self . autoscale_view ( ) 
 3897 

 /databricks/python/local/lib/python2.7/site-packages/matplotlib/axes/_base.pyc in add_collection (self, collection, autolim) 
 1672 
 1673 if autolim : 
 -> 1674 self . update_datalim ( collection . get_datalim ( self . transData ) ) 
 1675 
 1676 collection . _remove_method = lambda h : self . collections . remove ( h ) 

 /databricks/python/local/lib/python2.7/site-packages/matplotlib/collections.pyc in get_datalim (self, transData) 
 194 transOffset = transOffset . get_affine ( ) 
 195 
 --> 196 offsets = np . asanyarray ( offsets , np . float_ ) 
 197 if np . ma . isMaskedArray ( offsets ) : 
 198 offsets = offsets . filled ( np . nan ) 

 /databricks/python/local/lib/python2.7/site-packages/numpy/core/numeric.pyc in asanyarray (a, dtype, order) 
 531 
 532 """
 --> 533 return array ( a , dtype , copy = False , order = order , subok = True ) 
 534 
 535 def ascontiguousarray ( a , dtype = None ) : 

 ValueError : setting an array element with a sequence.